In [3]:
import pandas as pd

global_df = pd.DataFrame()

def data_preprocessing(df):
    df = df.drop(list(range(22)))
    df = df.iloc[:, [0, 4]]
    df.columns = ['Date', 'Debit']

    df = df.dropna().reset_index(drop=True)

    df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)

    df['Year'] = df['Date'].dt.year
    df['Month'] = df['Date'].dt.month

    monthly_expenditure = df.groupby(['Year', 'Month'])['Debit'].sum().reset_index()
    monthly_expenditure.columns = ['Year', 'Month', 'Monthly_Expenditure']
    
    return monthly_expenditure

def update_data(new_data):

    global global_df

    new_preprocessed = data_preprocessing(new_data)

    global_df = pd.concat([global_df, new_preprocessed], ignore_index=True)
    
    return global_df 



In [4]:
df = pd.read_excel('2020 dummy bank.xls',header =None)
update_data(df)

,Year,Month,Monthly_Expenditure
0,2020,1,4339.88
1,2020,2,6047.58
2,2020,3,4779.25
3,2020,4,3876.47
4,2020,5,3404.87
5,2020,6,3862.84
6,2020,7,5406.35
7,2020,8,3383.04
8,2020,9,2236.56
9,2020,10,13083.92


In [5]:
df2 = pd.read_excel('2021 dummy bank.xls',header =None)
update_data(df2)

,Year,Month,Monthly_Expenditure
0,2020,1,4339.88
1,2020,2,6047.58
2,2020,3,4779.25
3,2020,4,3876.47
4,2020,5,3404.87
5,2020,6,3862.84
6,2020,7,5406.35
7,2020,8,3383.04
8,2020,9,2236.56
9,2020,10,13083.92


In [6]:
df3 = pd.read_excel('2022 dummy bank.xls',header =None)
update_data(df3)

,Year,Month,Monthly_Expenditure
0,2020,1,4339.88
1,2020,2,6047.58
2,2020,3,4779.25
3,2020,4,3876.47
4,2020,5,3404.87
5,2020,6,3862.84
6,2020,7,5406.35
7,2020,8,3383.04
8,2020,9,2236.56
9,2020,10,13083.92


In [12]:
import pandas as pd
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
import calendar

def predict():
    model = XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)
    
    predictions = {}
    true_values = []
    predicted_values = []

    latest_year = global_df['Year'].max()
    prediction_year = latest_year + 1

    for month in range(1, 13):
        month_data = global_df[global_df['Month'] == month]
        if month_data.empty:
            continue
        
        X = month_data[['Year']]
        y = month_data['Monthly_Expenditure']
        
        model.fit(X, y)
        
        future_year = pd.DataFrame({'Year': [prediction_year]})
        prediction = model.predict(future_year)[0]
        
        predictions[month] = prediction
        predicted_values.append(prediction)
        
        if latest_year in month_data['Year'].values:
            true_value = month_data[month_data['Year'] == latest_year]['Monthly_Expenditure'].values[0]
            true_values.append(true_value)

    predictions_df = pd.DataFrame(list(predictions.items()), columns=['Month', 'Predicted_Monthly_Expenditure'])
    predictions_df['Month_Name'] = predictions_df['Month'].apply(lambda x: calendar.month_name[x])
    predictions_df = predictions_df[['Month', 'Month_Name', 'Predicted_Monthly_Expenditure']]

    '''if true_values:
        mae = mean_absolute_error(true_values, predicted_values)
        r2 = r2_score(true_values, predicted_values)
        print(f'Mean Absolute Error: {mae}')
        print(f'R-squared: {r2}')
    else:
        print("Insufficient data for evaluation.")'''
    
    return predictions_df

In [13]:
predict()

,Month,Month_Name,Predicted_Monthly_Expenditure
0,1,January,3387.170410
1,2,February,6471.465820
2,3,March,4006.470215
3,4,April,10651.293945
4,5,May,3994.741211
5,6,June,3455.259521
6,7,July,4082.068604
7,8,August,6041.940918
8,9,September,4165.744141
9,10,October,5949.330078
